In [ ]:
# default_exp esm1b

# Colab library installation

So it's honestly a bit annoying to go through the whole song and dance of installing the necessary dependencies on collab, when in reality all the dependencies are defined in a conda environment file.. Nevertheless, for now, I'm just trying to explore the hugging face implementation of ESM, so supposedly, I'd only need to install hugging face???

In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 33.8 MB/s 
     |████████████████████████████████| 7.6 MB 75.9 MB/s 
     |████████████████████████████████| 182 kB 15.3 MB/s 


In [2]:
#hide
from nbdev.showdoc import *

ModuleNotFoundError: ignored

# Load libraries

In [5]:
from transformers import EsmTokenizer, EsmForMaskedLM
import torch

In [140]:
import pandas as pd

## Loading different models

Now that ESM _potentially_ has a bunch of different models, it is probably best to have some sort of functional way of loading the different models, instead of globally loading the models like a crazy person..

In [64]:
def load_model(model_path, tokenizerLM, maskedLM):
  tokenizer = tokenizerLM.from_pretrained(model_path)
  model = maskedLM.from_pretrained(model_path)
  return tokenizer, model

In [68]:
esm1b_tokenizer, esm1b_model = load_model("facebook/esm1b_t33_650M_UR50S",EsmTokenizer,EsmForMaskedLM)

Downloading:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/782 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/esm1b_t33_650M_UR50S were not used when initializing EsmForMaskedLM: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
- This IS expected if you are initializing EsmForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [82]:
esm2_tokenizer, esm2_model = load_model("facebook/esm2_t6_8M_UR50D",EsmTokenizer,EsmForMaskedLM)

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForMaskedLM: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
- This IS expected if you are initializing EsmForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
inputs = tokenizer("MEN<mask>EL", return_tensors="pt")
inputs

{'input_ids': tensor([[ 0, 20,  9, 17, 32,  9,  4,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [77]:
def run_model(model, inputs):
  with torch.no_grad():
    logits = model(**inputs).logits
  return logits

In [78]:
esm2_logits = run_model(esm2_model, inputs)

In [71]:
def logits2prob(logits):
  return torch.softmax(logits,dim=2)

In [79]:
esm2_prob = logits2prob(esm2_logits)
esm2_prob

tensor([[[9.9995e-01, 9.8169e-11, 3.3333e-09, 1.0032e-10, 2.7753e-06,
          2.1860e-06, 2.8457e-06, 1.7699e-06, 2.4734e-06, 3.8847e-06,
          1.8188e-06, 1.4733e-06, 8.3154e-07, 1.3514e-06, 5.0538e-07,
          2.6128e-06, 1.6407e-06, 5.7511e-07, 1.0479e-06, 7.2024e-07,
          6.8558e-06, 4.7543e-07, 2.9435e-07, 4.1178e-06, 5.8838e-06,
          2.6893e-11, 1.9237e-11, 1.2725e-11, 5.7512e-13, 3.3849e-13,
          4.0601e-13, 3.2575e-13, 9.8609e-11],
         [4.9910e-07, 8.4403e-10, 6.0393e-07, 8.3639e-10, 1.6646e-03,
          4.8789e-04, 7.7675e-04, 1.0370e-03, 8.0296e-04, 9.2551e-04,
          5.9806e-04, 6.1500e-04, 1.0765e-03, 8.0351e-04, 5.8132e-04,
          1.0121e-03, 5.3399e-04, 7.9683e-04, 5.6664e-04, 3.9000e-04,
          9.8611e-01, 3.3409e-04, 1.6016e-04, 3.7677e-04, 3.5096e-04,
          1.6065e-08, 1.0686e-08, 6.3483e-09, 2.7585e-10, 1.8574e-10,
          1.9849e-10, 1.6479e-10, 8.4401e-10],
         [4.9243e-06, 6.5413e-09, 1.0006e-05, 6.5484e-09, 5.8789e-

In [58]:
with torch.no_grad():
    logits = model(**inputs).logits
logits

tensor([[[ 1.2999e+01, -1.0045e+01, -6.5200e+00, -1.0023e+01,  2.0458e-01,
          -3.4107e-02,  2.2962e-01, -2.4525e-01,  8.9383e-02,  5.4085e-01,
          -2.1800e-01, -4.2872e-01, -1.0007e+00, -5.1505e-01, -1.4986e+00,
           1.4422e-01, -3.2108e-01, -1.3694e+00, -7.6945e-01, -1.1444e+00,
           1.1089e+00, -1.5597e+00, -2.0392e+00,  5.9913e-01,  9.5600e-01,
          -1.1340e+01, -1.1675e+01, -1.2088e+01, -1.5185e+01, -1.5715e+01,
          -1.5533e+01, -1.5753e+01, -1.0041e+01],
         [-7.8156e+00, -1.4198e+01, -7.6250e+00, -1.4207e+01,  2.9671e-01,
          -9.3057e-01, -4.6553e-01, -1.7652e-01, -4.3235e-01, -2.9031e-01,
          -7.2696e-01, -6.9904e-01, -1.3919e-01, -4.3167e-01, -7.5535e-01,
          -2.0083e-01, -8.4027e-01, -4.4001e-01, -7.8093e-01, -1.1545e+00,
           6.6809e+00, -1.3093e+00, -2.0445e+00, -1.1890e+00, -1.2600e+00,
          -1.1252e+01, -1.1659e+01, -1.2180e+01, -1.5316e+01, -1.5712e+01,
          -1.5645e+01, -1.5831e+01, -1.4198e+01],


In [26]:
logits.shape

torch.Size([1, 8, 33])

In [23]:
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
mask_token_index

tensor([4])

In [24]:
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

'L'

Awesome, this is starting to definitely look similar in vibe to what I remember from the protBERT hugging face implementation. Luckily.. I already have some nice functions to work with that so I should be able to generalize between the two approaches. Honestly, it's a bit of a question of whether protBERT is relevant to the updated ESM models, and also if ESM1B can be loaded from huggingface.

## Test out protBERT berteome functions

I have a few functions that does some pretty helpful things with the hugging face implementation that I want to test out here. I'm just going to copy and paste here as I see fit, so that I can start to tweak them as needed.

Dame, so I may need to regroup since some of the inference functions are a bit different?? Definitely need to try to make this a bit more generalizable?? I think part of it could be hugging face versions or maybe just a BERT huggingface thing that isn't present in the ESM approach. Honestly, the inference doesn't look to different from before??

# Updated ESM approach

In [141]:
class modelPredDF():
    def __init__(self, predDict, seq, aas):
        self.predDf = pd.DataFrame.from_dict(predDict, orient = "index", columns = list(aas))
        self.predDf = self.predDf.div(self.predDf.sum(axis=1),axis=0)
        self.predDf.insert(0, "wt",list(seq))
        self.predDf.insert(1, "wtIndex",list(range(1,len(seq)+1)))
        wtScore = self.wtScoreCol()
        self.predDf.insert(2, "wtScore",wtScore)

    def wtScoreCol(self):
        wtScore = []
        for row in self.predDf.to_dict(orient="records"):
	        wt = row["wt"]
	        wtScore.append(row[wt])
        return wtScore

In [39]:
naturalAAs = "ACDEFGHIKLMNPQRSTVWY"

In [53]:
tokenizer(naturalAAs)["input_ids"][1:-1]

[5, 23, 13, 9, 18, 6, 21, 12, 15, 4, 20, 17, 14, 16, 10, 8, 11, 7, 22, 19]

In [128]:
def naturalAAIndex(aas, tokenizer):
    return tokenizeSeq(aas, tokenizer, return_tensors=None)["input_ids"][1:-1]

In [137]:
def getNatProbs(natAAList,probList):
    natProbList = []
    for natAAIndex in natAAList:
      natProbList.append(probList[natAAIndex])
    return natProbList

In [129]:
naturalAAIndex(naturalAAs,esm2_tokenizer)

[5, 23, 13, 9, 18, 6, 21, 12, 15, 4, 20, 17, 14, 16, 10, 8, 11, 7, 22, 19]

In [120]:
naturalAAIndex(naturalAAs,esm2_tokenizer)["input_ids"]

[tensor([ 0,  5, 23, 13,  9, 18,  6, 21, 12, 15,  4, 20, 17, 14, 16, 10,  8, 11,
          7, 22, 19,  2])]

In [108]:
def maskifySeq(seq, i):
    seqList = list(seq)
    if i != None:
      seqList[i] = "<mask>" # it would be nice if this could somewhow be derived from the model library..
    return ''.join(seqList)

In [105]:
maskifySeq("MENDEL",3)

'MEN<mask>EL'

In [124]:
def tokenizeSeq(seq, tokenizer, mask_index = None, return_tensors = "pt"):
  maskified_seq = maskifySeq(seq, mask_index)
  return tokenizer(maskified_seq, return_tensors=return_tensors)

In [125]:
tokenizeSeq("MENDEL",esm2_tokenizer)

{'input_ids': tensor([[ 0, 20,  9, 17, 13,  9,  4,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [126]:
mendel_mask3 = tokenizeSeq("MENDEL",esm2_tokenizer, mask_index = 3)

In [127]:
run_model(esm2_model, mendel_mask3)

tensor([[[ 1.2999e+01, -1.0045e+01, -6.5200e+00, -1.0023e+01,  2.0458e-01,
          -3.4107e-02,  2.2962e-01, -2.4525e-01,  8.9383e-02,  5.4085e-01,
          -2.1800e-01, -4.2872e-01, -1.0007e+00, -5.1505e-01, -1.4986e+00,
           1.4422e-01, -3.2108e-01, -1.3694e+00, -7.6945e-01, -1.1444e+00,
           1.1089e+00, -1.5597e+00, -2.0392e+00,  5.9913e-01,  9.5600e-01,
          -1.1340e+01, -1.1675e+01, -1.2088e+01, -1.5185e+01, -1.5715e+01,
          -1.5533e+01, -1.5753e+01, -1.0041e+01],
         [-7.8156e+00, -1.4198e+01, -7.6250e+00, -1.4207e+01,  2.9671e-01,
          -9.3057e-01, -4.6553e-01, -1.7652e-01, -4.3235e-01, -2.9031e-01,
          -7.2696e-01, -6.9904e-01, -1.3919e-01, -4.3167e-01, -7.5535e-01,
          -2.0083e-01, -8.4027e-01, -4.4001e-01, -7.8093e-01, -1.1545e+00,
           6.6809e+00, -1.3093e+00, -2.0445e+00, -1.1890e+00, -1.2600e+00,
          -1.1252e+01, -1.1659e+01, -1.2180e+01, -1.5316e+01, -1.5712e+01,
          -1.5645e+01, -1.5831e+01, -1.4198e+01],


In [143]:
seq = "MENDEL"
naturalAAIndices = naturalAAIndex("ACDEFGHIKLMNPQRSTVWY",esm2_tokenizer)
esmPredDict = {}
for wtIndex in range(len(seq)):
  maskedSeq = tokenizeSeq(seq, esm2_tokenizer, mask_index = wtIndex)
  #print(maskedSeq)
  seq_logits = run_model(esm2_model, maskedSeq)
  seq_probs = logits2prob(seq_logits)
  esmPredDict[wtIndex] = [i.item() for i in getNatProbs(naturalAAIndices, seq_probs[0, wtIndex +1])]
modelPredDF(esmPredDict, seq, "ACDEFGHIKLMNPQRSTVWY").predDf

,wt,wtIndex,wtScore,A,C,D,E,F,G,H,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.552697,0.021572,0.007771,0.037974,0.037202,0.016749,0.031286,0.008931,...,0.552697,0.025167,0.016633,0.015696,0.021180,0.027951,0.019174,0.032738,0.005038,0.015784
1,E,2,0.062526,0.053799,0.021036,0.043922,0.062526,0.046101,0.053995,0.025246,...,0.022747,0.046137,0.038938,0.035488,0.060355,0.072753,0.057780,0.065278,0.016475,0.031191
2,N,3,0.033663,0.061583,0.019181,0.059015,0.090463,0.040464,0.070657,0.020765,...,0.024444,0.033663,0.040042,0.030041,0.059641,0.060723,0.045562,0.077274,0.017029,0.025385
3,D,4,0.036863,0.044991,0.023413,0.036863,0.065857,0.045495,0.057128,0.025325,...,0.027285,0.039678,0.045763,0.041703,0.055303,0.073471,0.052912,0.059173,0.019147,0.031549
4,E,5,0.049126,0.044737,0.028491,0.045022,0.049126,0.059680,0.052702,0.032531,...,0.020457,0.045267,0.040136,0.036020,0.052129,0.079066,0.051003,0.063749,0.016661,0.038395
5,L,6,0.084846,0.047930,0.016511,0.053397,0.092545,0.040752,0.061778,0.021220,...,0.024157,0.055584,0.029879,0.034910,0.062608,0.059308,0.050503,0.062796,0.016001,0.030007


In [146]:
def esmPredictionDF(seq, tokenizer, model, aas = "ACDEFGHIKLMNPQRSTVWY"):
  naturalAAIndices = naturalAAIndex(aas,tokenizer)
  esmPredDict = {}
  for wtIndex in range(len(seq)):
    maskedSeq = tokenizeSeq(seq, tokenizer, mask_index = wtIndex)
    seq_logits = run_model(model, maskedSeq)
    seq_probs = logits2prob(seq_logits)
    esmPredDict[wtIndex] = [i.item() for i in getNatProbs(naturalAAIndices, seq_probs[0, wtIndex +1])]
  esmPredDF = modelPredDF(esmPredDict, seq, aas).predDf
  return esmPredDF

In [147]:
esmPredictionDF("MENDEL", esm2_tokenizer, esm2_model)

,wt,wtIndex,wtScore,A,C,D,E,F,G,H,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.552697,0.021572,0.007771,0.037974,0.037202,0.016749,0.031286,0.008931,...,0.552697,0.025167,0.016633,0.015696,0.021180,0.027951,0.019174,0.032738,0.005038,0.015784
1,E,2,0.062526,0.053799,0.021036,0.043922,0.062526,0.046101,0.053995,0.025246,...,0.022747,0.046137,0.038938,0.035488,0.060355,0.072753,0.057780,0.065278,0.016475,0.031191
2,N,3,0.033663,0.061583,0.019181,0.059015,0.090463,0.040464,0.070657,0.020765,...,0.024444,0.033663,0.040042,0.030041,0.059641,0.060723,0.045562,0.077274,0.017029,0.025385
3,D,4,0.036863,0.044991,0.023413,0.036863,0.065857,0.045495,0.057128,0.025325,...,0.027285,0.039678,0.045763,0.041703,0.055303,0.073471,0.052912,0.059173,0.019147,0.031549
4,E,5,0.049126,0.044737,0.028491,0.045022,0.049126,0.059680,0.052702,0.032531,...,0.020457,0.045267,0.040136,0.036020,0.052129,0.079066,0.051003,0.063749,0.016661,0.038395
5,L,6,0.084846,0.047930,0.016511,0.053397,0.092545,0.040752,0.061778,0.021220,...,0.024157,0.055584,0.029879,0.034910,0.062608,0.059308,0.050503,0.062796,0.016001,0.030007


# Previous ESM approach

In [3]:
# export
import torch
import esm
import pandas as pd

ModuleNotFoundError: ignored

In [ ]:
# export
from berteome import berteomeDF

In [ ]:
# export
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

In [ ]:
# export
batch_converter = alphabet.get_batch_converter()

In [ ]:
# export
def naturalAAIndex(aas):
    
    model_alphabet=alphabet.all_toks
    naturalAAIndices = []
    for aa in aas:
      naturalAAIndices.append(model_alphabet.index(aa))
    return naturalAAIndices

In [ ]:
assert naturalAAIndex("ACDEFGHIKLMNPQRSTVWY") == [5, 23, 13, 9, 18, 6, 21, 12, 15, 4, 20, 17, 14, 16, 10, 8, 11, 7, 22, 19]

In [ ]:
# export
def maskifySeq(seq, i):
    seqList = list(seq)
    seqList[i] = "<mask>"
    return seqList

In [ ]:
# export
def getNatProbs(natAAList,probList):
    natProbList = []
    for natAAIndex in natAAList:
      natProbList.append(probList[natAAIndex])
    return natProbList

In [ ]:
# export
def esmPredictionDF(seq, aas="ACDEFGHIKLMNPQRSTVWY"): # this should probably be a class where model/alphabet are declared??
    naturalAAIndices=naturalAAIndex(aas)
    esmPredDict = {}
    for wtIndex in range(len(seq)):
      maskSeq = maskifySeq(seq, wtIndex) # import from other module, ie seqtools.maskifySeq(), or not, maybe both classes should just be in the same module...
      data = [("protein1", maskSeq)]
      batch_labels, batch_strs, batch_tokens = batch_converter(data)
      with torch.no_grad():
        results = model(batch_tokens)
        logits = results['logits']
        prob = torch.softmax(logits,dim=2)
      esmPredDict[wtIndex] = [i.item() for i in getNatProbs(naturalAAIndices, prob[0, wtIndex +1])]
    esmPredDF = berteomeDF.modelPredDF(esmPredDict,seq, aas).predDf
    
    return esmPredDF

In [ ]:
esmPredictionDF("MENDEL")

,wt,wtIndex,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.033952,0.007039,0.054737,0.063365,0.018590,0.029267,0.010216,0.028865,...,0.456082,0.029166,0.023344,0.019043,0.023603,0.030170,0.023352,0.033982,0.004791,0.013076
1,E,2,0.058999,0.021031,0.054622,0.072057,0.037186,0.053090,0.025848,0.065879,...,0.024329,0.059710,0.035728,0.040369,0.052300,0.073618,0.057319,0.062975,0.014029,0.030339
2,N,3,0.055626,0.014060,0.077887,0.122153,0.034727,0.058822,0.019171,0.061401,...,0.031318,0.044172,0.029668,0.034335,0.052372,0.057394,0.047432,0.069432,0.013282,0.025859
3,D,4,0.044610,0.017627,0.036298,0.071025,0.031629,0.049782,0.023518,0.061833,...,0.058515,0.043762,0.037227,0.054164,0.055120,0.060091,0.052912,0.070125,0.017567,0.027211
4,E,5,0.045966,0.027927,0.047431,0.057574,0.052450,0.054545,0.030670,0.071293,...,0.025417,0.051106,0.033450,0.038940,0.055194,0.076423,0.050060,0.063550,0.017663,0.038091
5,L,6,0.048226,0.016231,0.060284,0.100391,0.031534,0.052584,0.023015,0.065281,...,0.029354,0.063840,0.025546,0.039313,0.062937,0.064147,0.053790,0.060177,0.013714,0.028147
